# API OpenAI - Dados de posts manifestações

In [3]:
# imports
from openai import OpenAI
import os
import pandas as pd
from groq import Groq
import openpyxl

In [ ]:
# api key
#client = OpenAI(api_key="MY_OPENAI_API_KEY")
client = Groq(api_key="MY_API_KEY")

In [32]:
# alterar PATH para o caminho do arquivo Posts_2013.xlsx
# data = pd.read_excel(r'G:\Drives compartilhados\LEMEP-Manchetometro\Livro Junho 2013\Textos Jornais 2013\Capas\FSP_2003-2013\manchetes_FSP_agregado_4.0.xlsx')
data = pd.read_excel("/Users/joaoferes/Library/CloudStorage/GoogleDrive-jferes@iesp.uerj.br/Shared drives/LEMEP-Manchetometro/Livro Junho 2013/Textos Jornais 2013/Capas/FSP_2003-2013/manchetes_FSP_agregado.xlsx")
data.head(10)

,Arquivo,Data,Manchete
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês
5,FSP_2010-01-06_0001.jpg,2010-01-06,Ponte desaba em rodovia do Sul; dez pessoas de...
6,FSP_2010-01-07_0001.jpg,2010-01-07,"Prejuízo com chuva supera R$ 1,1 bi"
7,FSP_2010-01-08_0001.jpg,2010-01-08,Jobim fará seu próprio relatório sobre os acags
8,FSP_2010-01-09_0001.jpg,2010-01-09,Ministro critica plano de direitos humanos
9,FSP_2010-01-10_0001.jpg,2010-01-10,Vannuchi ameaça demissão se plano punir tortur...


In [33]:
len(data)

3866

In [34]:
# funcao para usar o modelo preenchendo respostas
def use_model(question, text):
    completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role":"system", "content":"Você é um assistente que analisa manchetes de jornal e responde perguntas a respeito."},
        {"role":"user", "content":f"Manchete:{text}\n\nPergunta:{question}"}
    ]
)
    return completion.choices[0].message.content

In [35]:
# adicionar colunas novas (inicialmente vazias) para as respostas
data["Menção ao Lula"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Valência Lula"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,,
...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",,


In [36]:
# definindo as perguntas 1 e 2

pergunta_1 = """A manchete menciona o Lula?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional."""

pergunta_2 = """Qual é a valência da menção ao Lula?

Responda com apenas "Favorável", "Contrária" ou "Neutra", sem qualquer texto adicional.

Sendo que:
- Favorável: a menção ao Lula é posita, citando dados ou fatos positivos sobre ele ou seu governo. 
- Contrária: a menção ao Lula é negativa, citando dados ou fatos negativos sobre ele ou seu governo.
- Neutra: a menção ao Lula não é nem positiva nem negativa, apenas informativa.
"""

In [37]:
# for cada linha no dataset
    # montando uma string com todas as informacoes importantes do post que queremos passar pra ia 
    # usando a funcao use_model para responder a pergunta no post atual
    # adicionando a resposta no dataset

for index, row in data.iterrows():    
    #if index <= 3559: 
    #    continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_1 = use_model(pergunta_1, manchete)
    data.loc[index, "Menção ao Lula"] = answer_1 # precisa mudar todas vez que for adicionar variável nova
    # answer_2 = use_model(pergunta_2, manchete)
    # data.loc[index, "Valência Lula"] = answer_2 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 1: {answer_1}")

Index: 0 Manchete: Chuva mata ao menos 19 no RJ Resposta 1: Não
Index: 1 Manchete: Soterraamentos matam 30 em Angra Resposta 1: Não
Index: 2 Manchete: Empresários vão priorizar doações ocultas na eleição Resposta 1: Não
Index: 3 Manchete: Inaugurações dominam ano eleitoral Resposta 1: Não
Index: 4 Manchete: FAB prefere caça sueco a francês Resposta 1: Não
Index: 5 Manchete: Ponte desaba em rodovia do Sul; dez pessoas desaparecem Resposta 1: Não
Index: 6 Manchete: Prejuízo com chuva supera R$ 1,1 bi Resposta 1: Não
Index: 7 Manchete: Jobim fará seu próprio relatório sobre os acags Resposta 1: Não
Index: 8 Manchete: Ministro critica plano de direitos humanos Resposta 1: Não
Index: 9 Manchete: Vannuchi ameaça demissão se plano punir torturadores Resposta 1: Não
Index: 10 Manchete: Governo estuda excluir de plano menção a torturador Resposta 1: Não
Index: 11 Manchete: Gasolina terá menos álcool para segurar alta de preço Resposta 1: Não
Index: 12 Manchete: Forte terremoto atinge o Haiti Re

In [38]:
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,Não,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,Não,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,Não,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,Não,
...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,Não,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,Não,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",Não,


In [39]:
data_mencao_Lula = data[data["Menção ao Lula"] == "Sim"] # precisa mudar todas vez que for adicionar variável nova
data_mencao_Lula

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula
25,FSP_2010-01-26_0001.jpg,2010-01-26,"Na frente de Serra, Lula propõe PAC contra enc...",Sim,
27,FSP_2010-01-28_0001.jpg,2010-01-28,Lula ignora TCU e dá verba para obras sob susp...,Sim,
29,FSP_2010-01-30_0001.jpg,2010-01-30,Chávez terá ajuda de Lula contra crise de energia,Sim,
31,FSP_2010-02-01_0001.jpg,2010-02-01,Governo de Lula acelera criação de cargos no J...,Sim,
55,FSP_2010-02-25_0001.jpg,2010-02-25,Cuba reprime protesto da oposição na visita de...,Sim,
...,...,...,...,...,...
3660,FSP_2011-06-09_0001.jpg,2011-06-09,"Dilma demite Palocci na hora certa, diz Lula",Sim,
3752,FSP_2011-09-09_0001.jpg,2011-09-09,Valério questiona falta de Lula como réu do me...,Sim,
3803,FSP_2011-10-30_0001.jpg,2011-10-30,Lula tem câncer na laringe e vai passar por qu...,Sim,
3822,FSP_2011-11-18_0001.jpg,2011-11-18,Lula recebe diária por viagem com agenda parti...,Sim,


In [40]:
for index, row in data_mencao_Lula.iterrows():    
    manchete = row["Manchete"]
    
    # usar o modelo para responder a pergunta
    answer_2 = use_model(pergunta_2, manchete)
    data.loc[index, "Valência Lula"] = answer_2 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta: {answer_2}")


Index: 25 Manchete: Na frente de Serra, Lula propõe PAC contra enchente Resposta: Favorável
Index: 27 Manchete: Lula ignora TCU e dá verba para obras sob suspeita Resposta: Contrária
Index: 29 Manchete: Chávez terá ajuda de Lula contra crise de energia Resposta: Favorável
Index: 31 Manchete: Governo de Lula acelera criação de cargos no Judiciário Resposta: Neutra
Index: 55 Manchete: Cuba reprime protesto da oposição na visita de Lula Resposta: Neutra
Index: 68 Manchete: Lula compara presos políticos de Cuba aos bandidos de SP Resposta: Contrária
Index: 71 Manchete: Lula admite que Jefferson o alertou sobre mensalão Resposta: Contrária
Index: 74 Manchete: Israelenses cobram de Lula distância do Irã Resposta: Contrária
Index: 78 Manchete: Serra assume candidatura com elogio ao governo Lula Resposta: Favorável
Index: 79 Manchete: Lula inaugura obras que voltam a ser grandes cantores Resposta: Favorável
Index: 86 Manchete: Popularidade de Lula bate recorde Resposta: Favorável
Index: 103 Ma

In [41]:
# Inserindo as variáveis Governo Federal e Valência Governo Federal
# adicionar colunas novas (inicialmente vazias) para as respostas
data["Governo Federal"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Valência Governo Federal"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,Não,,,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,Não,,,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,,,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,Não,,,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,Não,,,
...,...,...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,Não,,,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,,,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,Não,,,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",Não,,,


In [42]:
# definindo as perguntas 3 e 4

pergunta_3 = """A manchete menciona o Governo Federal?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional.
Menções à palavra Governo devem ser consideradas positivas, mesmo que não utilizem o adjetivo "Federal"."""

pergunta_4 = """Qual é a valência da menção ao Governo Federal?

Responda com apenas "Favorável", "Contrária" ou "Neutra", sem qualquer texto adicional.

Sendo que:
- Favorável: a menção ao Governo Federal é posita, citando dados ou fatos positivos sobre seus resultados e políticas públicas. 
- Contrária: a menção ao Governo Federal é negativa, citando dados ou fatos negativos sobre seus resultados e políticas públicas.
- Neutra: a menção ao Governo Federal não é nem positiva nem negativa, apenas informativa.
"""

In [46]:
# for cada linha no dataset
    # montando uma string com todas as informacoes importantes do post que queremos passar pra ia 
    # usando a funcao use_model para responder a pergunta no post atual
    # adicionando a resposta no dataset

for index, row in data.iterrows():    
    if index <= 2995: 
        continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_3 = use_model(pergunta_3, manchete)
    data.loc[index, "Governo Federal"] = answer_3 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 3: {answer_3}")

print("Processamento finalizado")

Index: 2996 Manchete: PF prende alunos da Unifesp após protestos em SP Resposta 3: Não
Index: 2997 Manchete: Erundina diz que apoio de Maluf ao PT sair caro ao partido Resposta 3: Não
Index: 2998 Manchete: CULPADOS Resposta 3: Não
Index: 2999 Manchete: Haddad começa 2º turno 10 pontos à frente de Serra Resposta 3: Não
Index: 3000 Manchete: Bancos privados cedem à Dilma e a barateiam crédito Resposta 3: Sim
Index: 3001 Manchete: Escuta mostra que senador negociou verba para a Delta Resposta 3: Não
Index: 3002 Manchete: Greve de policiais e bombeiros leva medo a Fortaleza Resposta 3: Não
Index: 3003 Manchete: PM promete fim do tráfico em 30 dias na cracolândia Resposta 3: Não
Index: 3004 Manchete: Com crise política e CPI, Dilma abre cofre a aliados Resposta 3: Sim
Index: 3005 Manchete: França elege presidente socialista com discurso de conciliação, François Hollande vence Sarkozy Resposta 3: Não
Index: 3006 Manchete: CUT ameaça ir às ruas em defesa de réus Resposta 3: Não
Index: 3007 Ma

In [47]:
data_mencao_GF = data[data["Governo Federal"] == "Sim"] 
data_mencao_GF

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal
8,FSP_2010-01-09_0001.jpg,2010-01-09,Ministro critica plano de direitos humanos,Não,,Sim,
10,FSP_2010-01-11_0001.jpg,2010-01-11,Governo estuda excluir de plano menção a tortu...,Não,,Sim,
16,FSP_2010-01-17_0001.jpg,2010-01-17,"Brasil ficará no Haiti mais 5 anos, diz Jobim",Não,,Sim,
20,FSP_2010-01-21_0001.jpg,2010-01-21,Brasil vai dobrar tropas para o Haiti,Não,,Sim,
27,FSP_2010-01-28_0001.jpg,2010-01-28,Lula ignora TCU e dá verba para obras sob susp...,Sim,Contrária,Sim,
...,...,...,...,...,...,...,...
3845,FSP_2011-12-11_0001.jpg,2011-12-11,Receita deve aliviar regra para as declarações...,Não,,Sim,
3848,FSP_2011-12-14_0001.jpg,2011-12-14,"Penas do mensalão vão prescrever, diz ministro",Não,,Sim,
3851,FSP_2011-12-17_0001.jpg,2011-12-17,Dilma rejeita interferência de partidos no gov...,Não,,Sim,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,,Sim,


In [49]:
for index, row in data_mencao_GF.iterrows():    
    if index <= 3804: 
        continue
    manchete = row["Manchete"]
    
    # usar o modelo para responder a pergunta
    answer_4 = use_model(pergunta_4, manchete)
    data.loc[index, "Valência Governo Federal"] = answer_4 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta: {answer_4}")


Index: 3811 Manchete: Premié sai para a Grécia formar governo de união Resposta: Neutra
Index: 3819 Manchete: Ministro aprova 7 sindicatos fantasmas Resposta: Contrária
Index: 3826 Manchete: Egípcios voltam às ruas, e governo pede para sair Resposta: Contrária
Index: 3827 Manchete: Governo estuda distribuir parte dos recursos do FGTS Resposta: Neutra
Index: 3836 Manchete: Crise faz governo acelerar pacote para ativar economia Resposta: Neutra
Index: 3838 Manchete: Situação de Lupi é insustentável, avisa o governo Resposta: Contrária
Index: 3839 Manchete: Lupi é sexto ministro de Dilma a cair sob suspeita Resposta: Contrária
Index: 3840 Manchete: Governo vai acabar com entrada de IR de empresa Resposta: Favorável
Index: 3845 Manchete: Receita deve aliviar regra para as declarações de IR Resposta: Favorável
Index: 3848 Manchete: Penas do mensalão vão prescrever, diz ministro Resposta: Contrária
Index: 3851 Manchete: Dilma rejeita interferência de partidos no governo Resposta: Neutra
Inde

In [50]:
data["PT"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Valência PT"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,Não,,Não,,,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,Não,,Não,,,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,,Não,,,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,Não,,Não,,,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,Não,,Não,,,
...,...,...,...,...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,Não,,Não,,,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,,Sim,Contrária,,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,Não,,Não,,,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",Não,,Não,,,


In [51]:
# definindo as perguntas 5 e 6

pergunta_5 = """A manchete menciona o PT?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional.
Menções a pessoas importantes do partido, como Lula, Dilma ou outros deve ser consideradas como menções ao PT."""

pergunta_6 = """Qual é a valência da menção ao PT?

Responda com apenas "Favorável", "Contrária" ou "Neutra", sem qualquer texto adicional.

Sendo que:
- Favorável: a menção ao PT é posita, citando dados ou fatos positivos sobre seus resultados e políticas públicas. 
- Contrária: a menção ao PT é negativa, citando dados ou fatos negativos sobre seus resultados e políticas públicas.
- Neutra: a menção ao PT não é nem positiva nem negativa, apenas informativa.
"""

In [58]:
# Rodando código PT (pergunta 5)

for index, row in data.iterrows():    
    if index <= 3865: 
        continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_5 = use_model(pergunta_5, manchete)
    data.loc[index, "PT"] = answer_5 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 5: {answer_5}")

In [59]:
data_mencao_PT = data[data["PT"] == "Sim"] 
data_mencao_PT

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT
25,FSP_2010-01-26_0001.jpg,2010-01-26,"Na frente de Serra, Lula propõe PAC contra enc...",Sim,Favorável,Não,,Sim,
27,FSP_2010-01-28_0001.jpg,2010-01-28,Lula ignora TCU e dá verba para obras sob susp...,Sim,Contrária,Sim,Contrária,Sim,
29,FSP_2010-01-30_0001.jpg,2010-01-30,Chávez terá ajuda de Lula contra crise de energia,Sim,Favorável,Sim,Favorável,Sim,
31,FSP_2010-02-01_0001.jpg,2010-02-01,Governo de Lula acelera criação de cargos no J...,Sim,Neutra,Sim,Favorável,Sim,
46,FSP_2010-02-16_0001.jpg,2010-02-16,"Para Dilma, Estado deve ser também 'empresário'",Não,,Sim,Favorável,Sim,
...,...,...,...,...,...,...,...,...,...
3758,FSP_2011-09-15_0001.jpg,2011-09-15,Governo Dilma pode perder 5º ministro,Não,,Sim,Contrária,Sim,
3803,FSP_2011-10-30_0001.jpg,2011-10-30,Lula tem câncer na laringe e vai passar por qu...,Sim,Neutra,Não,,Sim,
3822,FSP_2011-11-18_0001.jpg,2011-11-18,Lula recebe diária por viagem com agenda parti...,Sim,Contrária,Não,,Sim,
3839,FSP_2011-12-05_0001.jpg,2011-12-05,Lupi é sexto ministro de Dilma a cair sob susp...,Não,,Sim,Contrária,Sim,


In [ ]:
for index, row in data_mencao_PT.iterrows(): 
    #if index <= 3865: 
    #    continue   
    manchete = row["Manchete"]
    
    # usar o modelo para responder a pergunta
    answer_6 = use_model(pergunta_6, manchete)
    data.loc[index, "Valência PT"] = answer_6 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta: {answer_6}")


In [62]:
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,Não,,Não,,Não,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,Não,,Não,,Não,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,,Não,,Não,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,Não,,Não,,Não,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,Não,,Não,,Não,
...,...,...,...,...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,Não,,Não,,Não,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,,Sim,Contrária,Não,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,Não,,Não,,Não,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",Não,,Não,,Não,


In [63]:
# definindo as perguntas 7 e 8

pergunta_7 = """A manchete menciona o tema da corrupção?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional.
Menções a investigações, condenações, processos judiciais contra políticos, ou mesmo suspeitas de práticas de corrupção contam como "sim". 
Menções a corrupção em geral, sem citar casos específicos, também contam como "sim".
"""

pergunta_8 = """No texto, Há qualquer menção que relaciona o PT à corrupção?

Responda com apenas "Sim" ou "Não", sem qualquer texto adicional.

Menções à corrupção que involvam pessoas importantes do PT, como Lula, Dilma, Del[ubio, Palocci, Vacari, Dirceu ou outros devem ser consideradas como menções a corrupção do PT, e portanto, "sim".
"""

In [64]:
data["Corrupção"] = "" # cada variável adicionada modifica o valor entre colchetes
data["Corrupção PT"] = ""
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT,Corrupção,Corrupção PT
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,Não,,Não,,Não,,,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,Não,,Não,,Não,,,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,,Não,,Não,,,
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,Não,,Não,,Não,,,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,Não,,Não,,Não,,,
...,...,...,...,...,...,...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,Não,,Não,,Não,,,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,,Sim,Contrária,Não,,,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,Não,,Não,,Não,,,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",Não,,Não,,Não,,,


In [67]:
# Rodando código Corrupçao (pergunta 7)

for index, row in data.iterrows():    
    if index <= 3317: 
        continue
    manchete = row["Manchete"]
    # usar o modelo para responder a pergunta
    answer_7 = use_model(pergunta_7, manchete)
    data.loc[index, "Corrupção"] = answer_7 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta 5: {answer_7}")

Index: 3318 Manchete: Após réuaste, Petrobras despenca até 10% na Bolsa Resposta 5: Sim
Index: 3319 Manchete: Qualidade do serviço cai em SP Resposta 5: Não
Index: 3320 Manchete: Da para fazer muito mais que BC, diz Campos Resposta 5: Não
Index: 3321 Manchete: Auditoria mostra que governo paga preços 'distorcidos' Resposta 5: Sim
Index: 3322 Manchete: PF prende 22 por suspeita de fraudes em minibolsas Resposta 5: Sim
Index: 3323 Manchete: Formando de medicina terá de trabalhar 2 anos no SUS Resposta 5: Não
Index: 3324 Manchete: Itamaraty cobra explicação dos EUA sobre espionagem Resposta 5: Não
Index: 3325 Manchete: Kassab arquivou apuração contra mim, diz fiscal Resposta 5: Sim
Index: 3326 Manchete: Cortes no Senado e poupam regalias e auxiliar de check-in Resposta 5: Sim
Index: 3327 Manchete: Frase de Dilma sobre a inflação afeta a relação com o PMDB Resposta 5: Não
Index: 3328 Manchete: Com tarifa menor, Eletrobras atinge prejuízo recorde Resposta 5: Não
Index: 3329 Manchete: Índice

In [68]:
data_mencao_Corrup = data[data["Corrupção"] == "Sim"] 
data_mencao_Corrup

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT,Corrupção,Corrupção PT
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,,Não,,Não,,Sim,
7,FSP_2010-01-08_0001.jpg,2010-01-08,Jobim fará seu próprio relatório sobre os acags,Não,,Não,,Não,,Sim,
23,FSP_2010-01-24_0001.jpg,2010-01-24,SP investiga 800 delegados,Não,,Não,,Não,,Sim,
24,FSP_2010-01-25_0001.jpg,2010-01-25,Câmara rasa gasto secreto com doador,Não,,Não,,Não,,Sim,
27,FSP_2010-01-28_0001.jpg,2010-01-28,Lula ignora TCU e dá verba para obras sob susp...,Sim,Contrária,Sim,Contrária,Sim,Contrária,Sim,
...,...,...,...,...,...,...,...,...,...,...,...
3848,FSP_2011-12-14_0001.jpg,2011-12-14,"Penas do mensalão vão prescrever, diz ministro",Não,,Sim,Contrária,Não,,Sim,
3854,FSP_2011-12-20_0001.jpg,2011-12-20,Supremo reduz poder do CNJ e investigar juízes,Não,,Não,,Não,,Sim,
3855,FSP_2011-12-21_0001.jpg,2011-12-21,Ministro do STF deu liminar que o beneficia,Sim,Favorável,Não,,Não,,Sim,
3856,FSP_2011-12-22_0001.jpg,2011-12-22,CNJ inspeciona 217 mil e abre guerra no Judici...,Não,,Não,,Não,,Sim,


In [72]:
for index, row in data_mencao_Corrup.iterrows():    
    if index <= 3209: 
        continue
    manchete = row["Manchete"]
    
    # usar o modelo para responder a pergunta
    answer_8 = use_model(pergunta_8, manchete)
    data.loc[index, "Corrupção PT"] = answer_8 # precisa mudar todas vez que for adicionar variável nova
    # if index == 100:
    #     break
    
    print(f"Index: {index}", f"Manchete: {manchete}", f"Resposta: {answer_8}")

print("Processamento concluído")



Index: 3210 Manchete: PT deve esquecer mensalão, diz ex-ministro de Lula Resposta: Sim
Index: 3212 Manchete: Para relator, é desnecessário CPI convocar procurador Resposta: Não
Index: 3214 Manchete: TSE tira da eleição político que teve contas rejeitadas Resposta: Não
Index: 3219 Manchete: Mensalão: para relator, José Dirceu comandava a quadrilha Resposta: Sim
Index: 3221 Manchete: Lula incentiva, e Congresso anuncia CPI do Cachoeira Resposta: Não
Index: 3225 Manchete: Acusação é afonta à lei, diz defesa de José Dirceu Resposta: Sim
Index: 3226 Manchete: Dilma demite assessora por suspeita de corrupção Resposta: Sim
Index: 3227 Manchete: Assessoria da Presidência em SP é suspeita de corrupção Resposta: Não
Index: 3228 Manchete: Análise do mensalão vai durar até 2 anos, diz ministro do STF Resposta: Não
Index: 3229 Manchete: Serra sugeriu que Siemens fizesse acordo, diz e-mail Resposta: Não
Index: 3230 Manchete: STF muda posição e diz que cassação cabe ao Congresso Resposta: Não
Index: 

In [73]:
data

,Arquivo,Data,Manchete,Menção ao Lula,Valência Lula,Governo Federal,Valência Governo Federal,PT,Valência PT,Corrupção,Corrupção PT
0,FSP_2010-01-01_0001.jpg,2010-01-01,Chuva mata ao menos 19 no RJ,Não,,Não,,Não,,Não,
1,FSP_2010-01-02_0001.jpg,2010-01-02,Soterraamentos matam 30 em Angra,Não,,Não,,Não,,Não,
2,FSP_2010-01-03_0001.jpg,2010-01-03,Empresários vão priorizar doações ocultas na e...,Não,,Não,,Não,,Sim,Não
3,FSP_2010-01-04_0001.jpg,2010-01-04,Inaugurações dominam ano eleitoral,Não,,Não,,Não,,Não,
4,FSP_2010-01-05_0001.jpg,2010-01-05,FAB prefere caça sueco a francês,Não,,Não,,Não,,Não,
...,...,...,...,...,...,...,...,...,...,...,...
3861,FSP_2011-12-27_0001.jpg,2011-12-27,Homicídio cai e roubo de veículos sobe em SP,Não,,Não,,Não,,Não,
3862,FSP_2011-12-28_0001.jpg,2011-12-28,Faculdades cobram R$ 500 milhões de bolsa do g...,Não,,Sim,Contrária,Não,,Não,
3863,FSP_2011-12-29_0001.jpg,2011-12-29,Tráfego para o interior de SP sobe mais que pa...,Não,,Não,,Não,,Não,
3864,FSP_2011-12-30_0001.jpg,2011-12-30,"Em 2011, ouro e dólar foram os mais rentáveis",Não,,Não,,Não,,Não,


In [ ]:
data.to_csv(r'G:\Drives compartilhados\LEMEP-Manchetometro\Livro Junho 2013\Textos Jornais 2013\Capas\ESP_2003-2013\manchetes_ESP_teste_4.csv', sep=';', index=False) # cuidado para não sobreescrever

In [76]:
data.to_excel("/Users/joaoferes/Library/CloudStorage/GoogleDrive-jferes@iesp.uerj.br/Shared drives/LEMEP-Manchetometro/Livro Junho 2013/Textos Jornais 2013/Capas/FSP_2003-2013/manchetes_FSP_agregado_cod.xlsx", index=False)